In [1]:
!pip install emoji
import emoji
import warnings
warnings.filterwarnings("ignore")

In [2]:
print(emoji.emojize(":angry_face:"))
print(emoji.emojize(":OK_hand_light_skin_tone:"))

😠
👌🏻


In [3]:
import pandas as pd
import numpy as np

In [4]:
data=pd.read_csv("train_emoji.csv" ,header = None)

In [5]:
data.head(n=10)

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN
5,I love you mum,0,NaN,NaN
6,Stop saying bullshit,3,NaN,NaN
7,congratulations on your acceptance,2,NaN,NaN
8,The assignment is too long,3,NaN,NaN
9,I want to go play,1,NaN,[3]


In [6]:
emoji_dict = {
    0:":beating_heart:",
    1:":baseball:",
    2:":beaming_face_with_smiling_eyes:",
    3:":disappointed_face:",
    4:":fork_and_knife:"    
}

In [7]:
for ix in emoji_dict.values():
    print(emoji.emojize(ix))

💓
⚾
😁
😞
🍴


In [8]:
X_train = data[0]
Y_train = data[1]

In [9]:
print(X_train.shape , Y_train.shape)

(132,) (132,)


In [10]:
for ix in range(132):
    X_train[ix] = X_train[ix].lower().split()

In [11]:
X_train[1]

['i', 'am', 'proud', 'of', 'your', 'achievements']

In [12]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r' , encoding = 'utf-8') as f:
        word = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            word.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:],dtype = np.float64)
        i = 1
        words_to_index ={}
        index_to_words ={}
        for w in sorted(word):
            words_to_index[w] = i
            index_to_words[i] = w
            i+=1
        return words_to_index , index_to_words , word_to_vec_map

In [13]:
words_to_index , index_to_words , word_to_vec_map = read_glove_vecs("glove.6B.50d.txt")

In [14]:
word_to_vec_map["i"]

array([ 1.1891e-01,  1.5255e-01, -8.2073e-02, -7.4144e-01,  7.5917e-01,
       -4.8328e-01, -3.1009e-01,  5.1476e-01, -9.8708e-01,  6.1757e-04,
       -1.5043e-01,  8.3770e-01, -1.0797e+00, -5.1460e-01,  1.3188e+00,
        6.2007e-01,  1.3779e-01,  4.7108e-01, -7.2874e-02, -7.2675e-01,
       -7.4116e-01,  7.5263e-01,  8.8180e-01,  2.9561e-01,  1.3548e+00,
       -2.5701e+00, -1.3523e+00,  4.5880e-01,  1.0068e+00, -1.1856e+00,
        3.4737e+00,  7.7898e-01, -7.2929e-01,  2.5102e-01, -2.6156e-01,
       -3.4684e-01,  5.5841e-01,  7.5098e-01,  4.9830e-01, -2.6823e-01,
       -2.7443e-03, -1.8298e-02, -2.8096e-01,  5.5318e-01,  3.7706e-02,
        1.8555e-01, -1.5025e-01, -5.7512e-01, -2.6671e-01,  9.2121e-01])

In [15]:
def avg_vector(words):
    ans  = word_to_vec_map[words[0]]
    for i in range(1, len(words)):
        ans = np.add(ans , word_to_vec_map[words[i]])
    ans = ans/float(len(words))
    return ans


In [16]:
print(avg_vector(X_train[1]))
print(X_train[1])

[ 0.00563117  0.59711333 -0.13939717 -0.384325    0.74846167 -0.34453833
 -0.09402833  0.13647617 -0.27891     0.07412743 -0.05458783  0.26998333
 -0.40014283 -0.29957133  0.5051755   0.13981333  0.06849917  0.53107067
  0.07820433 -0.29929733 -0.27029933  0.497221    0.24299683  0.03004167
  0.92919667 -1.6411     -1.137715    0.00513     0.459105   -0.29221167
  2.87481667  0.687305   -0.307625   -0.16341317 -0.2818225  -0.1250206
  0.139661    0.345859   -0.045965   -0.339121   -0.02768405 -0.36635967
 -0.164233    0.32527533 -0.164843    0.216988   -0.32414    -0.06701233
 -0.025773    0.32180167]
['i', 'am', 'proud', 'of', 'your', 'achievements']


In [17]:
X_numbers = np.zeros((132,50))
for i in  range(132):
    X_numbers[i] = avg_vector(X_train[i])

In [18]:
from keras.models import Sequential
from keras.layers import Dense , Input , Activation
from keras.utils import to_categorical

Using TensorFlow backend.


In [19]:
model = Sequential()

model.add(Dense(10,activation="relu" ,  input_shape=(50,)))
model.add(Dense(24,activation="relu"))
model.add(Dense(16,activation="relu"))
model.add(Dense(8,activation="relu"))
model.add(Dense(5 , activation="softmax"))

In [20]:
model.compile(optimizer="adam" , loss= "categorical_crossentropy" , metrics=["accuracy"])

In [21]:
ytrain = to_categorical(Y_train)


In [22]:
model.fit(X_numbers , ytrain , epochs=100)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/100
132/132 [==============================] - 0s 3ms/step - loss: 1.6346 - acc: 0.1667
Epoch 2/100
132/132 [==============================] - 0s 213us/step - loss: 1.6215 - acc: 0.1667
Epoch 3/100
132/132 [==============================] - 0s 179us/step - loss: 1.6097 - acc: 0.1667
Epoch 4/100
132/132 [==============================] - 0s 156us/step - loss: 1.6001 - acc: 0.1667
Epoch 5/100
132/132 [==============================] - 0s 137us/step - loss: 1.5930 - acc: 0.1667
Epoch 6/100
132/132 [==============================] - 0s 102us/step - loss: 1.5859 - acc: 0.1742
Epoch 7/100
132/132 [==============================] - 0s 113us/step - loss: 1.5784 - acc: 0.1894
Epoch 8/100
132/132 [==============================] - 0s 82us/step - loss: 1.5726 - acc: 0.2045
Epoch 9/100
132/132 [==============================] - 0s 111us/step - loss: 1.5665 - acc: 0.2273
Epoch 10/100
132/132 [====

132/132 [==============================] - 0s 152us/step - loss: 0.7392 - acc: 0.7727
Epoch 79/100
132/132 [==============================] - 0s 140us/step - loss: 0.7366 - acc: 0.7273
Epoch 80/100
132/132 [==============================] - 0s 107us/step - loss: 0.7133 - acc: 0.7500
Epoch 81/100
132/132 [==============================] - 0s 79us/step - loss: 0.6861 - acc: 0.8106
Epoch 82/100
132/132 [==============================] - 0s 82us/step - loss: 0.6649 - acc: 0.7955
Epoch 83/100
132/132 [==============================] - 0s 79us/step - loss: 0.6618 - acc: 0.7879
Epoch 84/100
132/132 [==============================] - 0s 194us/step - loss: 0.6401 - acc: 0.8106
Epoch 85/100
132/132 [==============================] - 0s 84us/step - loss: 0.6102 - acc: 0.8333
Epoch 86/100
132/132 [==============================] - 0s 162us/step - loss: 0.6054 - acc: 0.8258
Epoch 87/100
132/132 [==============================] - 0s 103us/step - loss: 0.5793 - acc: 0.8409
Epoch 88/100
132/132 [=====

In [23]:
output = np.argmax(model.predict(X_numbers) , axis=1)

In [24]:
print(Y_train[:10])

0    3
1    2
2    3
3    0
4    4
5    0
6    3
7    2
8    3
9    1
Name: 1, dtype: int64


In [25]:
output[:10]

array([0, 2, 3, 0, 4, 0, 3, 2, 0, 1])

In [26]:
text = "i hate cooking"
avg_text = avg_vector(text.split())
avg_v = avg_text.reshape(1,50)

In [27]:
emoji_predict = np.argmax(model.predict(avg_v))
print(emoji_predict)
print(emoji.emojize(emoji_dict[emoji_predict]))

4
🍴
